In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# from google.colab import drive
# drive.mount("/content/drive/")

path = 'drive/My Drive/Artificial intelligence/lab3/'

In [ ]:
class MnistNet(nn.Module):
    """
    Liteweight network architecture for the Mnist dataset (digit) classification
    """
    def __init__(self):
        super(MnistNet, self).__init__()
        self.num_classes = 10
        
        # fully convolutional part
        self.features = nn.Sequential(
            nn.Conv2d(1, 4, kernel_size=5),
            nn.BatchNorm2d(4),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(4, 4, kernel_size=5),
            nn.Dropout(p=0.2),
            nn.BatchNorm2d(4),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(inplace=True)  
        )
        
        # classifier, FC layers
        self.classifier = nn.Sequential(
            nn.Linear(16*4,16),
            nn.ReLU(inplace=True),
            nn.Linear(16,self.num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x.view(-1,x.size(-3)*x.size(-2)*x.size(-1)))
        return x


def train(model, train_loader, optimizer):
    """
    Training of an epoch
    model: network
    train_loader: train_loader loading images and labels in batches
    optimizer: optimizer to use in the training
    """
    model.train()
    total_loss = 0
    
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad() # zero the accumulated gradients
        output = model(data) # computer network's output
        loss = F.cross_entropy(output, target) # computer the loss
        loss.backward() # backward pass
        optimizer.step() # update weights
        
        total_loss = total_loss + loss.item()
        
        if batch_idx % 100 == 0:
            print('[{}/{} ({:.0f}%)]\tBatch loss: {:.6f}'.format(
                batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()/len(data)))
    
    print('Training: Epoch average loss {:.6f}'.format(total_loss/len(train_loader.dataset)))
         
        
def test(model, val_loader):
    """
    Compute accuracy on the validation set
    model: network
    val_loader: test_loader loading images and labels in batches
    """
    model.eval()
    
    # implement validation procedure, report accuracy on the validation set
    
    ### YOUR CODE HERE
    test_losses = []
    test_loss = 0
    correct = 0
    with torch.no_grad():
      for data, target in val_loader:
        output = model(data)
        test_loss += F.cross_entropy(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(val_loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(correct, len(val_loader.dataset), 100. * correct / len(val_loader.dataset)))


In [ ]:
# mnist dataset structure - train part
mnist_dataset_train = datasets.MNIST(path + 'drive/My Drive/Artificial intelligence/lab3/vs3ex1data/mnist_data', train=True, download=True, transform=transforms.Compose([
                   transforms.ToTensor(),
                   transforms.Normalize((0.1307,), (0.3081,))
               ]))
# mnist dataset structure - test part
mnist_dataset_val = datasets.MNIST('drive/My Drive/Artificial intelligence/lab3/vs3ex1data/mnist_data', train=False, download=True, transform=transforms.Compose([
                   transforms.ToTensor(),
                   transforms.Normalize((0.1307,), (0.3081,))
               ]))
mnist_dataset_val
# show sample images
print('Sample images')
for i in range(0,100,10):
    plt.imshow(Image.fromarray(mnist_dataset_train.train_data[i].numpy(), mode='L'))
    plt.show()

In [ ]:


# loader of the training set
train_loader = torch.utils.data.DataLoader(mnist_dataset_train,batch_size=16, shuffle=True)
# loader of the validation set
val_loader = torch.utils.data.DataLoader(mnist_dataset_val,batch_size=512, shuffle=False)

model = MnistNet() # initialize network structure

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20 + 1):
        print('Epoch {}'.format(epoch))
        train(model, train_loader, optimizer)
        test(model, val_loader)


By adding Batch Normalizations and one dropout layer after convolutions I increase the Accuracy on 2-3% in average (from 95% to 98%)
